## FewShotTemplate
- `FewShotTemplate`은 몇 개의 예시(Few-shot examples)를 제공하여 AI가 원하는 형태의 답변을 생성하도록 도와주는 도구입니다
- 예시를 통해 AI가 패턴을 학습하여 일관된 형식으로 답변할 수 있게 해줍니다
    - `examples` : 예시 데이터들의 리스트
    - `example_prompt` : 각 예시를 어떻게 표현할지 정의하는 템플릿
    - `prefix` : 예시들 앞에 들어갈 설명
    - `suffix` : 예시들 뒤에 들어갈 질문이나 지시사항


LLM 객체 만들기

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="midm-2.0-base-instruct-q5_k_m",
    temperature=0.7,
)


### 기본 FewShotPromptTemplate 사용법
- 군대 용어 설명하는 예시를 만들어보자

In [ ]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

# 군대 용어 설명 예시 데이터
examples = [
    {
        "word": "PX", 
        "definition": "Post Exchange의 줄임말로, 군부대 내에 있는 매점이나 편의시설을 말합니다."
    },
    {
        "word": "GOP", 
        "definition": "General Outpost의 줄임말로, 최전방 경계초소를 의미합니다."
    },
    {
        "word": "내무반", 
        "definition": "군인들이 생활하는 숙소로, 여러 명이 함께 지내는 공동생활 공간입니다."
    }
]

print("예시 데이터:")
for example in examples:
    print(f"- {example['word']}: {example['definition']}")


각 예시를 어떻게 표현할지 정의하는 `example_prompt` 만들기

In [ ]:
# 각 예시를 어떻게 보여줄지 정의하는 템플릿
example_prompt = PromptTemplate(
    input_variables=["word", "definition"],
    template="용어: {word}\n설명: {definition}"
)

# 예시 하나로 테스트해보기
test_example = examples[0]
formatted_example = example_prompt.format(**test_example)
print("포맷된 예시:")
print(formatted_example)


`FewShotPromptTemplate`을 만들어서 실제 활용해보기

In [ ]:
# FewShotPromptTemplate 생성
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="당신은 군대 용어 전문가입니다. 다음 예시들을 참고하여 새로운 군대 용어를 설명해주세요.",
    suffix="용어: {input}\n설명:",
    input_variables=["input"]
)

# 전체 프롬프트 확인
test_prompt = few_shot_prompt.format(input="부사관")
print("완성된 프롬프트:")
print(test_prompt)


실제로 LLM에 질문해보기


In [ ]:
# 여러 군대 용어들로 테스트
test_words = ["부사관", "전역", "면회", "포상휴가"]

for word in test_words:
    prompt = few_shot_prompt.format(input=word)
    response = llm.invoke(prompt)
    print(f"\n🔍 질문: {word}")
    print(f"📝 답변: {response.content}")
    print("-" * 40)


## ExampleSelector를 활용한 동적 예시 선택
- 입력과 의미적으로 유사한 예시들을 자동으로 선택하는 방법
- `SemanticSimilarityExampleSelector`는 벡터 유사도를 사용하여 가장 관련성 높은 예시들을 찾아줌
- 더 정확하고 맥락에 맞는 Few-shot 추론이 가능함

예시들을 임베딩 하기 위해 백터DB FAISS와 HuggingFace에서 임베딩 모델을 가져온다.

In [ ]:
%pip install langchain_community

In [ ]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# 더 많은 예시 데이터로 확장
extended_examples = [
    {"word": "PX", "definition": "Post Exchange의 줄임말로, 군부대 내에 있는 매점이나 편의시설을 말합니다."},
    {"word": "GOP", "definition": "General Outpost의 줄임말로, 최전방 경계초소를 의미합니다."},
    {"word": "내무반", "definition": "군인들이 생활하는 숙소로, 여러 명이 함께 지내는 공동생활 공간입니다."},
    {"word": "사격", "definition": "군사훈련의 하나로, 총기를 이용하여 목표물을 맞추는 훈련입니다."},
    {"word": "행군", "definition": "군인들이 무거운 장비를 메고 장거리를 걸어서 이동하는 훈련입니다."},
    {"word": "점호", "definition": "정해진 시간에 부대원들의 출석을 확인하고 공지사항을 전달하는 시간입니다."},
    {"word": "휴가", "definition": "군 복무 중 일정 기간 부대를 벗어나 개인적인 시간을 갖는 것입니다."},
    {"word": "면회", "definition": "가족이나 지인이 부대를 방문하여 군인과 만나는 것입니다."},
    {"word": "부사관", "definition": "중급 간부로서 부대의 중추적 역할을 담당하는 전문 군인입니다."},
    {"word": "장교", "definition": "부대를 지휘하고 관리하는 고급 간부를 의미합니다."}
]

임베딩 모델과 벡터 스토어를 이용한 `SemanticSimilarityExampleSelector` 설정

In [ ]:
# 임베딩 모델 설정 (한국어 지원)
embeddings = HuggingFaceEmbeddings(
    model_name="nlpai-lab/KURE-v1",  # 한국어 임베딩 모델
    model_kwargs={'device': 'cpu'}
)

# 의미적 유사도 기반 예시 선택기 생성
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=extended_examples,  # 예시 데이터들
    embeddings=embeddings,       # 임베딩 모델
    vectorstore_cls=FAISS,       # 벡터 스토어 클래스
    k=3,                         # 선택할 예시 개수
)

print("SemanticSimilarityExampleSelector 생성 완료!")
print(f"총 {len(extended_examples)}개 예시 중에서 상위 3개를 선택하도록 설정했습니다.")


선택된 예시들 확인해보기

In [ ]:
# 테스트할 군대 관련 용어들을 리스트로 정의
test_inputs = ["지휘관", "훈련", "외박", "매점"]

# 각 테스트 용어에 대해 의미적으로 유사한 예시들을 찾아서 출력
for test_input in test_inputs:
    # example_selector를 사용하여 입력 용어와 의미적으로 유사한 예시들을 선택
    selected_examples = example_selector.select_examples({"word": test_input})
    
    # 현재 테스트 중인 입력 용어 출력
    print(f"\n 입력: '{test_input}'")
    print("선택된 유사한 예시들:")
    
    # 선택된 예시들을 순서대로 출력
    # definition은 가독성을 위해 앞부분 40자만 출력
    for example in selected_examples:
        print(f"{example['word']}: {example['definition'][:40]}...")
    
    # 구분선 출력으로 각 테스트 케이스를 시각적으로 구분
    print("-" * 60)

`SemanticSimilarityExampleSelector`를 사용한 FewShotPromptTemplate 만들기

In [ ]:
# SemanticSimilarityExampleSelector를 사용한 FewShotPromptTemplate
semantic_few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # 의미적 유사도 선택기 사용
    example_prompt=example_prompt,
    prefix="당신은 군대 용어 전문가입니다. 다음 예시들을 참고하여 새로운 군대 용어를 설명해주세요.",
    suffix="용어: {input}\n설명:",
    input_variables=["input"]
)

# 테스트해보기
test_word = "지휘관"
semantic_prompt = semantic_few_shot_prompt.format(input=test_word)

# 완성된 프롬프트 출력력
print(semantic_prompt)


실제로 의미적 유사도 기반 Few-shot으로 LLM 테스트하기

In [ ]:
# 지휘관 용어로 테스트
word = "지휘관"

# 어떤 예시들이 선택되었는지 먼저 확인
selected = example_selector.select_examples({"word": word})

# 선택된 예시들에서 'word' 값만 추출하여 리스트로 만들기 (리스트 컴프리헨션 사용)
# ex는 각 예시 딕셔너리를 의미하고, ex['word']은 해당 예시의 용어명을 추출함
# 예: [{'word': 'PX', 'definition': '...'}, {'word': 'GOP', 'definition': '...'}] → ['PX', 'GOP']
selected_words = [ex['word'] for ex in selected]

# 프롬프트 생성 및 LLM 호출
prompt = semantic_few_shot_prompt.format(input=word)
response = llm.invoke(prompt)

print(f"\n질문: {word}")
print(f"참고 예시: {', '.join(selected_words)}")
print(f"답변: {response.content}")
print("=" * 80)
